In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import torch
import numpy as np
from PIL import Image
import os
import cv2
import random
import matplotlib.pyplot as plt
from torchvision import datasets,transforms

In [ ]:
data_path = '/content/drive/MyDrive/SpermWhaleData'

video1 = os.path.join(data_path, 'GX010047.MP4')
video2 = os.path.join(data_path, 'GX010068.MP4')
capture1 = cv2.VideoCapture(video1)
capture2 = cv2.VideoCapture(video2)


In [ ]:
try:
# creating a folder named data
    if not os.path.exists('data'):
        os.makedirs('data')

# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')

In [ ]:
#Video Frame Extraction
def videoFrameExtract(video,path_name):

  currentframe = 0

  while(True):

      # reading from frame
      ret,frame = video.read()

      if ret:
          # if video is still left continue creating images
          name = './data/'+ path_name +'/frame'+ str(currentframe) + '.jpg'
          try:
        # creating a folder named data
            if not os.path.exists('./data/'+ path_name):
                os.makedirs('./data/'+ path_name)

        # if not created then raise error
          except OSError:
              print ('Error: Creating directory of data')
          print ('Creating...' + name)

          # writing the extracted images
          cv2.imwrite(name, frame)

          # increasing counter so that it will
          # show how many frames are created
          currentframe += 1
      else:
          break

  # Release all space and windows once done
  video.release()
  cv2.destroyAllWindows()

videoFrameExtract(capture1,'video1')
videoFrameExtract(capture2,'video2')

Creating..../data/video1/frame0.jpg
Creating..../data/video1/frame1.jpg
Creating..../data/video1/frame2.jpg
Creating..../data/video1/frame3.jpg
Creating..../data/video1/frame4.jpg
Creating..../data/video1/frame5.jpg
Creating..../data/video1/frame6.jpg
Creating..../data/video1/frame7.jpg
Creating..../data/video1/frame8.jpg
Creating..../data/video1/frame9.jpg
Creating..../data/video1/frame10.jpg
Creating..../data/video1/frame11.jpg
Creating..../data/video1/frame12.jpg
Creating..../data/video1/frame13.jpg
Creating..../data/video1/frame14.jpg
Creating..../data/video1/frame15.jpg
Creating..../data/video1/frame16.jpg
Creating..../data/video1/frame17.jpg
Creating..../data/video1/frame18.jpg
Creating..../data/video1/frame19.jpg
Creating..../data/video1/frame20.jpg
Creating..../data/video1/frame21.jpg
Creating..../data/video1/frame22.jpg
Creating..../data/video1/frame23.jpg
Creating..../data/video1/frame24.jpg
Creating..../data/video1/frame25.jpg
Creating..../data/video1/frame26.jpg
Creating...

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gy2p1efx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gy2p1efx
  Resolved https://github.com/huggingface/transformers.git to commit ce4fff0be7f6464d713f7ac3e0bbaafbc6959ae5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8505457 sha256=1bd11a796de2411cdc938705a0156be6a45330d517b6b3d61b9bf5f7926c9474
  Stored in directory: /tmp/pip-ephem-wheel-cache-0wk8s1hb/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/954 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/99.2M [00:00<?, ?B/s]

In [ ]:
!rm -r ./output


rm: cannot remove './output': No such file or directory


In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import AutoModelForDepthEstimation, AutoImageProcessor
import time

image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf")

def depth_estimation(data, path, device, model, image_processor):
    currentframe = 0
    total_time = 0

    model.to(device)
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        for batch in data:
            # Move batch of images to the device
            batch = batch.to(device)

            # Prepare images for the model
            inputs = image_processor(images=batch, return_tensors="pt")
            inputs.to(device)

            # Measure the time taken for the forward pass
            start_time = time.time()
            outputs = model(**inputs)
            end_time = time.time()

            # Accumulate the total time taken
            total_time += end_time - start_time

            # Forward pass through the model
            predicted_depth = outputs.predicted_depth

            # Interpolate to original size for each image in the batch
            for i in range(len(batch)):
                img = batch[i]
                depth = predicted_depth[i]

                # Interpolate depth prediction to original size
                prediction = torch.nn.functional.interpolate(
                    depth.unsqueeze(0).unsqueeze(0),
                    size=(img.shape[1], img.shape[2]),
                    mode="bicubic",
                    align_corners=False,
                )

                # Convert depth prediction to numpy array
                depth_output = prediction.squeeze().cpu().numpy()

                # Save depth prediction
                name = './output/'+ path+ str(currentframe) + '.jpg'
                try:
                    # Creating a folder named 'output'
                    if not os.path.exists('./output/'+ path):
                        os.makedirs('./output/'+ path)
                except OSError:
                    print('Error: Creating directory of output')
                print('Creating...' + name)

                # Writing the depth prediction as an image
                cv2.imwrite(name, depth_output)

                # Print output after one forward pass and break
                print("Output after one forward pass:")

            # Print GPU memory usage
            print(f"GPU memory allocated after processing {len(batch)} images: {torch.cuda.memory_allocated(device) / 1024**2:.2f} MB")
            print(f"Max GPU memory allocated after processing {len(batch)} images: {torch.cuda.max_memory_allocated(device) / 1024**2:.2f} MB")

            # Break out of the loop after one forward pass
            break

    # Calculate the average time taken per batch
    avg_time_per_batch = total_time / len(batch)
    print(f"Average time per batch (batch size {len(batch)}): {avg_time_per_batch:.4f} seconds")


transform = transforms.Compose([
    transforms.ToTensor()
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dir, transform=None):
        self.data_dir = dir
        self.images = os.listdir(dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.data_dir, self.images[index])
        image = np.array(Image.open(image_path))

        if self.transform:
            image = self.transform(image)

        return image

for batch_size in [8, 16, 32, 64, 128, 256]:
    dataset = ImageDataset('./data/video1', transform)
    dataloader = torch.utils.data.DataLoader(dataset , batch_size=batch_size, shuffle=True)
    print("Batch size =", batch_size)
    device = torch.device('cuda')
    torch.cuda.empty_cache()
    depth_estimation(dataloader, path='./video1', device=device, model=model, image_processor=image_processor)

print(type(dataloader))
for index, img in enumerate(dataloader):
    print(index, img)


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!rm -r ./output

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import AutoModelForDepthEstimation, AutoImageProcessor
import time

image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-base-hf")
model = AutoModelForDepthEstimation.from_pretrained("LiheYoung/depth-anything-base-hf")

def depth_estimation(data, path, device, model, image_processor):
    currentframe = 0
    total_time = 0

    model.to(device)
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        for batch in data:
            # Move batch of images to the device
            batch = batch.to(device)

            # Prepare images for the model
            inputs = image_processor(images=batch, return_tensors="pt")
            inputs.to(device)

            # Measure the time taken for the forward pass
            start_time = time.time()
            outputs = model(**inputs)
            end_time = time.time()

            # Accumulate the total time taken
            total_time += end_time - start_time

            # Forward pass through the model
            predicted_depth = outputs.predicted_depth

            # Interpolate to original size for each image in the batch
            for i in range(len(batch)):
                img = batch[i]
                depth = predicted_depth[i]

                # Interpolate depth prediction to original size
                prediction = torch.nn.functional.interpolate(
                    depth.unsqueeze(0).unsqueeze(0),
                    size=(img.shape[1], img.shape[2]),
                    mode="bicubic",
                    align_corners=False,
                )

                # Convert depth prediction to numpy array
                depth_output = prediction.squeeze().cpu().numpy()

                # Save depth prediction
                name = './output/'+ path+ str(currentframe) + '.jpg'
                try:
                    # Creating a folder named 'output'
                    if not os.path.exists('./output/'+ path):
                        os.makedirs('./output/'+ path)
                except OSError:
                    print('Error: Creating directory of output')
                print('Creating...' + name)

                # Writing the depth prediction as an image
                cv2.imwrite(name, depth_output)

                # Print output after one forward pass and break
                print("Output after one forward pass:")

            # Print GPU memory usage
            print(f"GPU memory allocated after processing {len(batch)} images: {torch.cuda.memory_allocated(device) / 1024**2:.2f} MB")
            print(f"Max GPU memory allocated after processing {len(batch)} images: {torch.cuda.max_memory_allocated(device) / 1024**2:.2f} MB")

            # Break out of the loop after one forward pass
            break

    # Calculate the average time taken per batch
    avg_time_per_batch = total_time / len(batch)
    print(f"Average time per batch (batch size {len(batch)}): {avg_time_per_batch:.4f} seconds")


transform = transforms.Compose([
    transforms.ToTensor()
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dir, transform=None):
        self.data_dir = dir
        self.images = os.listdir(dir)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.data_dir, self.images[index])
        image = np.array(Image.open(image_path))

        if self.transform:
            image = self.transform(image)

        return image

for batch_size in [8, 16, 32, 64, 128, 256]:
    dataset = ImageDataset('./data/video1', transform)
    dataloader = torch.utils.data.DataLoader(dataset , batch_size=batch_size, shuffle=True)
    print("Batch size =", batch_size)
    device = torch.device('cuda')
    torch.cuda.empty_cache()
    depth_estimation(dataloader, path='./video1', device=device, model=model, image_processor=image_processor)

print(type(dataloader))
for index, img in enumerate(dataloader):
    print(index, img)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

Batch size = 8


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
GPU memory allocated after processing 8 images: 1232.77 MB
Max GPU memory allocated after processing 8 images: 6602.52 MB
Average time per batch (batch size 8): 0.3780 seconds
Batch size = 16
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video10.jpg
Output after one forward pass:
Creating..../output/./video1

OutOfMemoryError: CUDA out of memory. Tried to allocate 8.54 GiB. GPU 0 has a total capacty of 14.75 GiB of which 1.40 GiB is free. Process 24046 has 13.34 GiB memory in use. Of the allocated memory 13.17 GiB is allocated by PyTorch, and 36.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF